In [13]:
## ETL Project 2 extra database for future use

# Extract(optional,pull extended receipe data including sodium and sugar columns from spoonacular API  
# transform to df by pandas
# Load in pdadmin with database spoon_sugar with postgres, and save tables with sodium and sugar col, repectively

# key variations from main table: 
##Based on research healthy meal should consider maxSugar and maxSodium, which is 
##less than 15 and between 200-500(here 100 per meal is setted), repectively. As followed:

# maxSugar=25 
# maxSodium=100
# number=15
# random=true

In [34]:
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [35]:
api_Key= "384f494142cf48d19086f15bdb84231a" #use a config file 
url = "https://api.spoonacular.com/recipes/findByNutrients?maxSugar=25&maxSodium=100&number=15&random=true&apiKey=384f494142cf48d19086f15bdb84231a"
query_url = f'{url}&apiKey={api_Key}'
#response


In [36]:
# request parameters= {
#  recipe_id:
#  recipe_title:
#  recipe_sodium:
#  recipe_sugar:

# above is additional to main table below
    #     'minCarbs': 1,
    #     'maxCarbs': 1,
    #     'minProtein': 1,
    #     'maxProtein': 1,
    #     'minCalories': 1,
    #     'maxCalories': 1,
    #     'minFat': 1,
    #     'maxFat': 1,
    #     'minSugar': 1,
    #     'maxSugar': 1,
    #     'number': 15,
    #     'random': 'true'
    # 

In [37]:
counter = 0
recipe_id = []
recipe_title = []
recipe_sodium = []
recipe_sugar = []

# recipe_calories = []
# recipe_protein = []
# recipe_fat = []
# recipe_carbs = []

while counter < 500:
    query_url = f'{url}&apiKey={api_Key}'
    response = requests.get(query_url).json()
    
    for i in range(len(response)):
        try:
            recipe_id.append(response[0]['id'])
        except KeyError:
            recipe_id.append('null')
        recipe_title.append(response[0]['title'])
        recipe_sodium.append(response[0]['sodium'])
        recipe_sugar.append(response[0]['sugar'])
#       recipe_calories.append(response[0]['calories'])
#       recipe_protein.append(response[0]['protein'])
#       recipe_fat.append(response[0]['fat'])
#       recipe_carbs.append(response[0]['carbs'])
    counter += 1

KeyError: 0

In [ ]:
counter = 0
recipe_id = []
recipe_title = []
recipe_img_url = []
recipe_img_type = []
recipe_calories = []
recipe_protein = []
recipe_fat = []
recipe_carbs = []


while counter < 250:
    url = "https://api.spoonacular.com/recipes/findByNutrients?minCalories=25&maxCalories=1500&number=1&random=true&apiKey=1ae666d5198d469aafaffc3957a2b406"
    response = requests.get(url)
    result = response.json()
    
    for i in range(len(result)):
        try:
            recipe_id.append(result[0]['id'])
        except KeyError:
            recipe_id.append('null')
        recipe_title.append(result[0]['title'])
        recipe_img_url.append(result[0]['image'])
        recipe_img_type.append(result[0]['imageType'])
        recipe_calories.append(result[0]['calories'])
        recipe_protein.append(result[0]['protein'])
        recipe_fat.append(result[0]['fat'])
        recipe_carbs.append(result[0]['carbs'])


In [ ]:
cols = list(zip(recipe_id, recipe_title, recipe_sodium, recipe_sugar))
df = pd.DataFrame(cols, columns=['Spoonacular Recipe ID', "Recipe", 'Recipe Sodium', "Recipe Sugar"])

In [57]:
df.head()
len(df)
df.columns

Index(['spoonacular_recipe_id', 'recipe', 'recipe_sodium', 'recipe_sugar'], dtype='object')

In [39]:
df.to_csv("spoonacular_recipes_sugar.csv")

In [40]:
df = pd.read_csv ('spoonacular_recipes_sugar.csv')
print(df.nunique())

Unnamed: 0               0
Spoonacular Recipe ID    0
Recipe                   0
Recipe Sodium            0
Recipe Sugar             0
dtype: int64


In [58]:
df.head()


,spoonacular_recipe_id,recipe,recipe_sodium,recipe_sugar


In [42]:
df = df.drop('Unnamed: 0', 1)


C:\Users\adele\anaconda3\envs\PythonData1\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [43]:
#df.head()

In [52]:
df.drop_duplicates()
df = df.rename(columns={'Spoonacular Recipe ID': 'spoonacular_recipe_id',
                        'Recipe': 'recipe',
                        'Recipe Sodium': 'recipe_sodium',
                        'Recipe Sugar': 'recipe_sugar',
#                        'Recipe Calories': 'recipe_calories',
#                        'Recipe Protein Content (grams)': 'recipe_protein_content_grams',
#                        'Recipe Fat Content (grams)': 'recipe_fat_content_grams',
#                        'Recipe Carbs Content (grams)': 'recipe_carbs_content_grams'
                       })
df.head()

,spoonacular_recipe_id,recipe,recipe_sodium,recipe_sugar


In [53]:
rds_connection_string = "postgres:postgres@localhost:5432/spoon_sugar"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [54]:
engine.table_names()

C:\Users\adele\anaconda3\envs\PythonData1\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['spoon_sodium', 'spoon_sugar', 'spoonacular_recipes_sugar']

In [55]:
df.to_sql(name='spoonacular_recipes_sugar', con=engine, if_exists='append', index=False)